In [2]:
import numpy as np

# Problem setup
T = 1.0          # Terminal time
N = 100          # Number of time steps
dt = T / N       # Time step size
d = 1            # Dimension of Brownian motion
M = 10000        # Number of Monte Carlo samples


In [3]:
def sigma(t, x):
    return np.ones((M, d))  # Example: constant volatility

def sigma_inv(t, x):
    return np.ones((M, d))  # Inverse of sigma (identity for constant sigma)

def H(t, x, z):
    return -0.5 * np.sum(z**2, axis=1)  # Example Hamiltonian

def g(x):
    return np.sum(x**2-x, axis=1)  # Terminal condition


In [ ]:
def conditional_expectation(Y,X):
    # Simple Monte Carlo average as a placeholder for conditional expectation
    return np.mean(Y)

In [ ]:
class BSDE_Solver(object):
    def __init__(self, T, N, d, M):
        self.T = T
        self.N = N
        self.dt = T / N
        self.d = d
        self.M = M
        self.X = np.zeros((N + 1, M, d))
        self.Y = np.zeros((N + 1, M))
        self.Z = np.zeros((N, M, d))

    def simulate_brownian_motion(self):
        self.dW = np.sqrt(self.dt) * np.random.randn(self.M, self.N, self.d)
        W = np.cumsum(self.dW, axis=1)
        for n in range(N):
            self.X[n + 1] = self.X[n] + sigma(n * self.dt, self.X[n]) * self.dW[n]

    def solve(self):
        # Initial condition
        self.Y[:, -1] = g(self.X[:, -1, :])

        for n in reversed(range(self.N)):
            t_n = n * self.dt
            self.Z[:, n, :] = sigma_inv(t_n, self.X[:, n, :]) * (self.Y[:, n + 1] - self.Y[:, n]) / self.dt
            self.Y[:, n] = self.Y[:, n + 1] + H(t_n, self.X[:, n, :], self.Z[:, n, :]) * self.dt - np.sum(self.Z[:, n, :] * self.dW[:, n, :], axis=1)

In [5]:
X = np.zeros((N + 1, M, d))
Y = np.zeros((N + 1, M))
Z = np.zeros((N, M, d))
dW = np.random.normal(0, np.sqrt(dt), size=(N, M, d))


In [6]:
for n in range(N):
    X[n + 1] = X[n] + sigma(n * dt, X[n]) * dW[n]
X.shape

(101, 10000, 1)

In [7]:
Y[N] = g(X[N])
Y.shape

(101, 10000)

In [13]:
for n in reversed(range(N)):
    # Estimate conditional expectations using Monte Carlo
    EY = np.mean(Y[n + 1], axis=0)
    EYdW = np.mean(Y[n + 1][:, None] * dW[n], axis=0)

    # Compute Z and Y
    Z[n] = (1 / dt) * sigma_inv(n * dt, X[n]) * EYdW
    Y[n] = EY + H(n * dt, X[n], Z[n]) * dt


In [14]:
print("Approximate Y_0:", np.mean(Y[0]))


Approximate Y_0: 0.9866846575065882
